# Exploratory Data Analysis/Data Preparation/Data Exportation

#### The purpose of this notebook is to collect and clean data, perform preliminary eda and output dataframes as CSV for later combination in another notebook

#### Importing the necessary libraries

In [6]:
#DATA WRANGLING
import pandas as pd # Dataframes
from pandas.io.json import json_normalize # JSON wrangler
import statsapi # Python wrapper MLB data API

In [7]:
#DATA STORAGE
#from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

In [8]:
#DATA MANIPULATION AND MODELLING
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn_pandas import DataFrameMapper, FunctionTransformer, gen_features, pipeline
from sklearn_pandas.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
import xgboost as xgb
import os
import pitch_functions

In [9]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

## Data Extraction
Calling the StatsApi ```https://pypi.org/project/MLB-StatsAPI/``` to collect infomation about all MLB games played between 03/28/18 and 10/03/18. The schulde function will be used in order to collect specific game identifiers, those game identifiers will be used inside of a For Loop that will extract play-by-play data from each game and create a brand new data frame with game data.

Importing team codes from TSV file in ```public_data direcrtory```. Using this data to construct a data frame of each teams identifier for use later. 

In [10]:
teams = pd.read_csv('public_data/team_codes.tsv', sep='\t')

teams['full_name'] = teams['full_name'].str.lower().str.replace(' ', '_').str.replace('-','_')

In [11]:
teams.head()

,code,short_name,full_name
0,108,LAA,angels
1,109,ARI,d_backs
2,110,BAL,orioles
3,111,BOS,red_sox
4,112,CHC,cubs


In [16]:
team_code = []
for code, team_name in zip(teams['code'],teams['full_name']):
    text = (f'The {team_name} has code {code}')
    team_code.append(text)
team_code_df = pd.DataFrame(team_code)
team_code_df.head(1)

,0
0,The angels has code 108


In [199]:
schedule = statsapi.schedule(start_date="03/28/2018", end_date="10/03/2018")

In [200]:
full = json_normalize(schedule)
gamepks= full['game_id']

In [201]:
full.head(2)

,away_id,away_name,away_pitcher_note,away_probable_pitcher,away_score,current_inning,doubleheader,game_date,game_datetime,game_id,game_num,game_type,home_id,home_name,home_pitcher_note,home_probable_pitcher,home_score,inning_state,losing_pitcher,losing_team,save_pitcher,status,summary,winning_pitcher,winning_team
0,112,Chicago Cubs,Lester will make his 7th Opening Day start and...,"Lester, Jon",8,9,N,2018-03-29,2018-03-29T16:40:00Z,529407,1,R,146,Miami Marlins,"In 2017, Urena entered Spring Training out of ...","Urena, Jose",4,Bottom,Jose Urena,Miami Marlins,None,Final,2018-03-29 - Chicago Cubs (8) @ Miami Marlins ...,Steve Cishek,Chicago Cubs
1,134,Pittsburgh Pirates,Nova will make his first Opening Day start at ...,"Nova, Ivan",0,,N,2018-03-29,2018-03-29T17:10:00Z,529417,1,R,116,Detroit Tigers,Zimmermann makes his first Opening Day start a...,"Zimmermann, Jordan",0,,NaN,NaN,NaN,Postponed,2018-03-29 - Pittsburgh Pirates @ Detroit Tige...,NaN,NaN


In [202]:
gamepks_2018 = list(gamepks.unique())
len(gamepks_2018)

2435

In [203]:
test_pk = gamepks_2018

In [108]:
pd.set_option('display.max_columns', None)

## Data Frame Creation

This loop iterates through the game_pks from the above list, grabs the game data assoicated with each game pk and breaks it out into a custom human usuable pandas dataframe. The column names are defined manually so that only necessary columns are added to the data frame.
In the second loop, it iterates through play events in the current plays dataframe, this normalizes the ```.json``` nested in this data frame in the play events column. A dictionary is then defined, and several columns are added to the dictionary. This columns are used to add the prior pitch to the data frame. The prior pitch is important when determining patterns in pitcher tendencies. If/Else statements are then used to add each column and row to the dictionary which is then appended to a list, which is used to create a final dataframe.

In [204]:
list_for_final_df = []
for game in test_pk:
    curr_game = statsapi.get('game_playByPlay',{'gamePk':game})
    curr_plays = curr_game.get('allPlays')
    curr_plays_df = pd.DataFrame(curr_plays)
    curr_plays_norm = json_normalize(curr_plays)
    
    all_plays_cols = ['about.atBatIndex', 'about.halfInning', 'about.inning', 'count.balls', 'count.strikes', 'matchup.batSide.code', 
                     'matchup.batter.fullName', 'matchup.batter.id', 'matchup.pitchHand.code', 'matchup.splits.menOnBase', 'matchup.pitcher.fullName',
                     'matchup.pitcher.id', 'result.eventType']
    
    play_events_cols = ['count.balls', 'count.strikes', 'details.ballColor', 'details.call.code', 'details.call.description', 'details.type.description'
                        ,'details.call.code', 'details.description', 'details.code', 'details.type.code', 'index', 'pitchData.nastyFactor',
                       'pitchData.zone', 'pitchNumber', 'type']
    i = 1
    for index, row in curr_plays_norm.iterrows():
            play_events = json_normalize(row['playEvents'])
            
            for play_events_idx, play_events_row in play_events.iterrows():
                
                game_dict = {}
                game_dict['gamepk'] = game
                game_dict['pitch_id']  = str(game) + '_' + str(row['about.atBatIndex']) + '_' + str(i)
                game_dict['prior_pitch'] = str(game) + '_' + (str(row['about.atBatIndex']) + '_' + str(i - 1))
                
                
                for col_all_plays in all_plays_cols:
                    if col_all_plays in curr_plays_norm.columns:
                        game_dict[col_all_plays] = row[col_all_plays]
                    else:
                        game_dict[col_all_plays] = np.nan
                for col_play_events in play_events_cols:
                    if col_play_events in play_events.columns:
                        game_dict[col_play_events] = play_events_row[col_play_events]
                    else: 
                        game_dict[col_play_events] = np.nan
                
                list_for_final_df.append(game_dict)
                i += 1
                                                              
                                                              
                
                                                            
            
            

        
    


In [206]:
each_pitch = pd.DataFrame(list_for_final_df)
each_pitch.head(10)

,about.atBatIndex,about.halfInning,about.inning,count.balls,count.strikes,details.ballColor,details.call.code,details.call.description,details.code,details.description,details.type.code,details.type.description,gamepk,index,matchup.batSide.code,matchup.batter.fullName,matchup.batter.id,matchup.pitchHand.code,matchup.pitcher.fullName,matchup.pitcher.id,matchup.splits.menOnBase,pitchData.nastyFactor,pitchData.zone,pitchNumber,pitch_id,prior_pitch,result.eventType,type
0,0,top,1,0.0,0.0,"rgba(26, 86, 190, 1.0)",X,Hit Into Play - Out(s),E,"In play, run(s)",FT,Two-Seam Fastball,529407,0,L,Ian Happ,664023,R,Jose Urena,570632,Empty,32.89,6.0,1.0,529407_0_1,529407_0_0,home_run,pitch
1,1,top,1,1.0,0.0,"rgba(39, 161, 39, 1.0)",B,Ball - Called,B,Ball,FT,Two-Seam Fastball,529407,0,R,Kris Bryant,592178,R,Jose Urena,570632,Men_On,24.17,13.0,1.0,529407_1_2,529407_1_1,walk,pitch
2,1,top,1,2.0,0.0,"rgba(39, 161, 39, 1.0)",B,Ball - Called,B,Ball,FT,Two-Seam Fastball,529407,1,R,Kris Bryant,592178,R,Jose Urena,570632,Men_On,29.02,13.0,2.0,529407_1_3,529407_1_2,walk,pitch
3,1,top,1,2.0,1.0,"rgba(170, 21, 11, 1.0)",S,Strike - Swinging,S,Swinging Strike,FT,Two-Seam Fastball,529407,2,R,Kris Bryant,592178,R,Jose Urena,570632,Men_On,41.63,13.0,3.0,529407_1_4,529407_1_3,walk,pitch
4,1,top,1,3.0,1.0,"rgba(39, 161, 39, 1.0)",B,Ball - Called,B,Ball,CH,Changeup,529407,3,R,Kris Bryant,592178,R,Jose Urena,570632,Men_On,59.33,13.0,4.0,529407_1_5,529407_1_4,walk,pitch
5,1,top,1,3.0,2.0,"rgba(170, 21, 11, 1.0)",S,Strike - Swinging,F,Foul,FT,Two-Seam Fastball,529407,4,R,Kris Bryant,592178,R,Jose Urena,570632,Men_On,39.41,4.0,5.0,529407_1_6,529407_1_5,walk,pitch
6,1,top,1,4.0,2.0,"rgba(39, 161, 39, 1.0)",B,Ball - Called,B,Ball,CH,Changeup,529407,5,R,Kris Bryant,592178,R,Jose Urena,570632,Men_On,14.88,11.0,6.0,529407_1_7,529407_1_6,walk,pitch
7,2,top,1,0.0,1.0,"rgba(170, 21, 11, 1.0)",S,Strike - Swinging,F,Foul,FT,Two-Seam Fastball,529407,0,L,Anthony Rizzo,519203,R,Jose Urena,570632,RISP,45.62,9.0,1.0,529407_2_8,529407_2_7,hit_by_pitch,pitch
8,2,top,1,0.0,2.0,"rgba(170, 21, 11, 1.0)",S,Strike - Swinging,F,Foul,FT,Two-Seam Fastball,529407,1,L,Anthony Rizzo,519203,R,Jose Urena,570632,RISP,31.03,6.0,2.0,529407_2_9,529407_2_8,hit_by_pitch,pitch
9,2,top,1,1.0,2.0,"rgba(39, 161, 39, 1.0)",B,Ball - Called,B,Ball,FT,Two-Seam Fastball,529407,2,L,Anthony Rizzo,519203,R,Jose Urena,570632,RISP,53.95,12.0,3.0,529407_2_10,529407_2_9,hit_by_pitch,pitch


### NEXT STEPS:
- Add a previous pitch column 
- Scrape June 2018 and Sept 2018 hitting and pitching data
- Match the player stats to the name column in the pitch data
- Modify men on base 
- Drop play result and type column

    

In [207]:
pitch_id_df = each_pitch[['pitch_id', 'details.type.code']].copy()
pitch_id_df.head()

,pitch_id,details.type.code
0,529407_0_1,FT
1,529407_1_2,FT
2,529407_1_3,FT
3,529407_1_4,FT
4,529407_1_5,CH


In [208]:
merged_df = pd.merge(each_pitch, pitch_id_df,how='left', left_on='prior_pitch', right_on='pitch_id')

In [209]:
merged_df.head(2)

,about.atBatIndex,about.halfInning,about.inning,count.balls,count.strikes,details.ballColor,details.call.code,details.call.description,details.code,details.description,details.type.code_x,details.type.description,gamepk,index,matchup.batSide.code,matchup.batter.fullName,matchup.batter.id,matchup.pitchHand.code,matchup.pitcher.fullName,matchup.pitcher.id,matchup.splits.menOnBase,pitchData.nastyFactor,pitchData.zone,pitchNumber,pitch_id_x,prior_pitch,result.eventType,type,pitch_id_y,details.type.code_y
0,0,top,1,0.0,0.0,"rgba(26, 86, 190, 1.0)",X,Hit Into Play - Out(s),E,"In play, run(s)",FT,Two-Seam Fastball,529407,0,L,Ian Happ,664023,R,Jose Urena,570632,Empty,32.89,6.0,1.0,529407_0_1,529407_0_0,home_run,pitch,NaN,NaN
1,1,top,1,1.0,0.0,"rgba(39, 161, 39, 1.0)",B,Ball - Called,B,Ball,FT,Two-Seam Fastball,529407,0,R,Kris Bryant,592178,R,Jose Urena,570632,Men_On,24.17,13.0,1.0,529407_1_2,529407_1_1,walk,pitch,NaN,NaN


In [210]:
each_pitch_merged = merged_df

In [211]:
each_pitch_merged = each_pitch_merged.rename({'pitch_id_y': 'previous_pitch_in_ab', 'details.type.code_y': 'previous_pitch_code'}, axis=1)

In [212]:
each_pitch_merged.head(2)

,about.atBatIndex,about.halfInning,about.inning,count.balls,count.strikes,details.ballColor,details.call.code,details.call.description,details.code,details.description,details.type.code_x,details.type.description,gamepk,index,matchup.batSide.code,matchup.batter.fullName,matchup.batter.id,matchup.pitchHand.code,matchup.pitcher.fullName,matchup.pitcher.id,matchup.splits.menOnBase,pitchData.nastyFactor,pitchData.zone,pitchNumber,pitch_id_x,prior_pitch,result.eventType,type,previous_pitch_in_ab,previous_pitch_code
0,0,top,1,0.0,0.0,"rgba(26, 86, 190, 1.0)",X,Hit Into Play - Out(s),E,"In play, run(s)",FT,Two-Seam Fastball,529407,0,L,Ian Happ,664023,R,Jose Urena,570632,Empty,32.89,6.0,1.0,529407_0_1,529407_0_0,home_run,pitch,NaN,NaN
1,1,top,1,1.0,0.0,"rgba(39, 161, 39, 1.0)",B,Ball - Called,B,Ball,FT,Two-Seam Fastball,529407,0,R,Kris Bryant,592178,R,Jose Urena,570632,Men_On,24.17,13.0,1.0,529407_1_2,529407_1_1,walk,pitch,NaN,NaN


In [213]:
each_pitch_clean = each_pitch_merged.drop(['result.eventType', 'type', 'pitch_id_x', 'previous_pitch_in_ab', 'prior_pitch', 'details.ballColor'], axis=1)

In [214]:
each_pitch_clean.head(2)

,about.atBatIndex,about.halfInning,about.inning,count.balls,count.strikes,details.call.code,details.call.description,details.code,details.description,details.type.code_x,details.type.description,gamepk,index,matchup.batSide.code,matchup.batter.fullName,matchup.batter.id,matchup.pitchHand.code,matchup.pitcher.fullName,matchup.pitcher.id,matchup.splits.menOnBase,pitchData.nastyFactor,pitchData.zone,pitchNumber,previous_pitch_code
0,0,top,1,0.0,0.0,X,Hit Into Play - Out(s),E,"In play, run(s)",FT,Two-Seam Fastball,529407,0,L,Ian Happ,664023,R,Jose Urena,570632,Empty,32.89,6.0,1.0,NaN
1,1,top,1,1.0,0.0,B,Ball - Called,B,Ball,FT,Two-Seam Fastball,529407,0,R,Kris Bryant,592178,R,Jose Urena,570632,Men_On,24.17,13.0,1.0,NaN


## Pitch Modification 
There are three types of pitches(fastballs, breaking balls, and change-ups). 
Hitters really only need to know which one of these types will be coming next. I think by narrowing the options, it will help the model make better predictions. 

In [215]:
pitch_dict = {'FF': 'Fastball'}

In [216]:
pitch_dict['FT'] = 'Fastball'
pitch_dict['FC']= 'Fastball'
pitch_dict['FS'] = 'Fastball'
pitch_dict['CH'] = 'Changeup'
pitch_dict['SI'] = 'Fastball'
pitch_dict['FT'] = 'Fastball'
pitch_dict['CU'] = 'Breaking_Ball'
pitch_dict['SL'] = 'Breaking_Ball'
pitch_dict['KC'] = 'Breaking_Ball'
pitch_dict['nan'] = 'NA'

In [217]:
each_pitch_clean['pitch_type'] = each_pitch_clean['details.type.code_x'].map(pitch_dict)

In [218]:
each_pitch_clean['prior_pitch_type'] = each_pitch_clean['previous_pitch_code'].map(pitch_dict)

In [219]:
each_pitch_clean = each_pitch_clean.drop(['details.type.code_x', 'details.type.description', 'details.code', 'gamepk', 'index', 'matchup.batter.id'],axis=1)

In [222]:
each_pitch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 794428 entries, 0 to 794427
Data columns (total 20 columns):
about.atBatIndex            794428 non-null int64
about.halfInning            794428 non-null object
about.inning                794428 non-null int64
count.balls                 777251 non-null float64
count.strikes               777251 non-null float64
details.call.code           725481 non-null object
details.call.description    725481 non-null object
details.description         794428 non-null object
matchup.batSide.code        794428 non-null object
matchup.batter.fullName     794428 non-null object
matchup.pitchHand.code      794428 non-null object
matchup.pitcher.fullName    794428 non-null object
matchup.pitcher.id          794428 non-null int64
matchup.splits.menOnBase    794428 non-null object
pitchData.nastyFactor       719410 non-null float64
pitchData.zone              719367 non-null float64
pitchNumber                 725481 non-null float64
previous_pitch_code 

In [221]:
each_pitch_clean.to_csv(r'public_data/standard_pitch.csv', index=False, sep=',', encoding='utf-8')

## Pipeline Preparation

In [128]:
pitch_clean = each_pitch_clean.dropna().copy()

In [129]:
# Lost about 600, but this is just a test
pitch_clean.head(2)

,about.atBatIndex,about.halfInning,about.inning,count.balls,count.strikes,details.call.code,details.call.description,details.description,matchup.batSide.code,matchup.batter.fullName,matchup.pitchHand.code,matchup.pitcher.fullName,matchup.pitcher.id,matchup.splits.menOnBase,pitchData.nastyFactor,pitchData.zone,pitchNumber,previous_pitch_code,pitch_type,prior_pitch_type
2,1,top,1,2.0,0.0,B,Ball - Called,Ball,R,Kris Bryant,R,Jose Urena,570632,Men_On,29.02,13.0,2.0,FT,Fastball,Fastball
3,1,top,1,2.0,1.0,S,Strike - Swinging,Swinging Strike,R,Kris Bryant,R,Jose Urena,570632,Men_On,41.63,13.0,3.0,FT,Fastball,Fastball


In [130]:
target = pitch_clean['pitch_type']

In [184]:
predictors = pitch_clean.drop(['pitch_type', 'matchup.pitcher.fullName', 'matchup.batter.fullName'], axis=1)

In [185]:
numerical_values = list(predictors['pitchNumber'])

In [186]:
num_features = list(predictors.select_dtypes(exclude='object'))

In [187]:
numeric_transformer = Pipeline(steps=[('keeper', None)])

In [188]:
categorical_value_df = predictors.drop(['pitchNumber'], axis=1)

In [189]:
categorial_values = list(categorical_value_df)

In [190]:
cat_features = list(predictors.select_dtypes(include='object'))

In [191]:
cat_transfomer = Pipeline(steps=[('onehot', OneHotEncoder())])

In [192]:
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, num_features),
                                              ('cat', cat_transfomer, cat_features)])

In [193]:
classifiers = [DecisionTreeClassifier(), RandomForestClassifier(n_estimators=100, max_depth=5), 
               GradientBoostingClassifier(n_estimators=100), xgb.XGBClassifier()]

## Model Creation

In [194]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, random_state=10)

In [197]:
#X_test[X_test['matchup.batter.fullName'].str.contains("Lucas Sims")==True]

In [196]:
for classifier in classifiers:
    #Intialize classifier pipeline
    clf1 = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', classifier)])
    clf1.fit(X_train, y_train)
    one_hot_names = list(clf1.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot'].get_feature_names())
    final_feats = num_features + one_hot_names
    # Print accuracy metrics for each model using pitch_functions.py
    print(classifier)
    print('\n')
    print('Training Metrics')
    pitch_functions.calc_acc_and_f1_score(y_train, clf1.predict(X_train))
    print('\n')
    print('Testing Metrics')
    pitch_functions.calc_acc_and_f1_score(y_test, clf1.predict(X_test))
    print('\n')
    

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


Training Metrics
Accuracy:1.000
F1-Score: 1.000
AUC: 1.000


Testing Metrics
Accuracy:0.627
F1-Score: 0.629
AUC: 0.646


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


Training Metrics
Accuracy:0.648
F1-Score: 0.547
AUC: 0.544


Testing Metrics
Accuracy:0.652
F1-Score: 0